In [1]:
# define imports
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
#import pandas as pd
#import PIL
import os, random

from glob import glob
from mpl_toolkits.axes_grid1 import ImageGrid
from IPython.display import display, Image
from IPython.core.display import HTML



import copy
#import cv2                

from io import open
import json
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os, os.path, random
from PIL import Image
import requests
import shutil
import time

import torch
from torch.autograd import Variable
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms


%matplotlib inline


# my imports

from os import listdir
from os.path import isfile, join, abspath
from style_transfer.stylize import transfer


In [20]:
# All images are resized to 224x224 and normalized
# Only training images receive further augmentation

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),    
}


In [21]:
data_dir = 'dogImages_style_transfer'
print(os.path.join(data_dir, 'valid'))


# we create some dictionaries
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'valid', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=5,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'valid', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid', 'test']}
print(dataset_sizes)
class_names = image_datasets['train'].classes
n_classes = len(class_names)

dogImages/valid
{'train': 59573, 'valid': 835, 'test': 836}


In [2]:
#import pandas as pd
import boto3
import sagemaker

In [3]:
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

In [4]:
import os

# this is the S3 data_dir name
data_dir = 'dogImages'
train_dir = 'train'

# set prefix, a descriptive name for a directory  
prefix = 'capstone-project'

# upload data for the first time; otherwise use the second expression (because the S3 upload takes some time)
#train_location = sagemaker_session.upload_data(os.path.join(data_dir), key_prefix=prefix)
train_location = 's3://{}/{}/'.format(bucket, prefix) 

In [15]:
# your import and estimator code, here
from sagemaker.pytorch import PyTorch

output_path = 's3://{}/{}'.format(bucket, prefix)

# note: this PyTorch estimator is specified with code that includes
# the preprocessing with the imgaug library and is trained on the style
# transferred images
source_dir = 'source_pytorch'

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir=source_dir,
                    role=role,
                    framework_version='1.1.0',
                    train_instance_count=1,
                    #train_instance_type='ml.c4.xlarge',
                    train_instance_type='ml.p2.xlarge',
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'epochs': 10
                    })


In [16]:
%%time

# Train your estimator on S3 training data
estimator.fit({'train': train_location})

2019-08-04 19:50:35 Starting - Starting the training job...
2019-08-04 19:50:39 Starting - Launching requested ML instances......
2019-08-04 19:51:41 Starting - Preparing the instances for training......
2019-08-04 19:52:57 Downloading - Downloading input data..........................................
2019-08-04 19:59:58 Training - Training image download completed. Training in progress.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-08-04 20:00:00,071 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-08-04 20:00:00,096 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-08-04 20:00:03,119 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-08-04 20:00:03,431 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2019-08-04 20:00:03,432 sagemaker-containers INFO     Gen

Using device cuda.
Get train data loader.
Epoch: 1 #011Training Loss: 26.674755 #011Validation Loss: 7.162729
Epoch: 2 #011Training Loss: 34.419785 #011Validation Loss: 7.321543
Epoch: 3 #011Training Loss: 37.806259 #011Validation Loss: 8.022751
Epoch: 4 #011Training Loss: 39.815794 #011Validation Loss: 7.985204
Epoch: 5 #011Training Loss: 41.511583 #011Validation Loss: 9.034338
Epoch: 6 #011Training Loss: 42.272636 #011Validation Loss: 9.284262
Epoch: 7 #011Training Loss: 43.557440 #011Validation Loss: 9.195464
Epoch: 8 #011Training Loss: 43.956678 #011Validation Loss: 9.054931
Epoch: 9 #011Training Loss: 44.419265 #011Validation Loss: 8.957233
Epoch: 10 #011Training Loss: 45.134310 #011Validation Loss: 8.717339
2019-08-05 00:59:00,875 sagemaker-containers INFO     Reporting training SUCCESS

2019-08-05 00:59:05 Uploading - Uploading generated training model
2019-08-05 01:00:26 Completed - Training job completed
Billable seconds: 18450
CPU times: user 34.1 s, sys: 1.46 s, total: 35.5 

In [ ]:
%%time

# importing PyTorchModel
from sagemaker.pytorch import PyTorchModel

print(estimator.model_data)

# Create a model from the trained estimator data
# And point to the prediction script
model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='1.1.0',
                     entry_point='predict.py',
                     source_dir=source_dir)


In [22]:
# ATTENTION: Please use the ml.t2.large instance here. Otherwise the deployment will fail because of resource constraints
# predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')
predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.large')



--------------------------------------------------------------------------------------!

In [22]:
def test_acc(test_set, predictor):
    ''' 
    This method calculates our accuracy metric. For this, it will send 
    batches of data to the endpoint and receive the predictions
    '''
    test_acc = 0.0
    running_corrects = 0

    for inputs, labels in test_set:
        print(labels)
        test_y_preds = predictor.predict(inputs)

        _, preds_tensor = torch.max(torch.from_numpy(test_y_preds), 1)
        preds = np.squeeze(preds_tensor.numpy()) if not use_cuda else np.squeeze(preds_tensor.cpu().numpy())

        print(f"preds is {preds}")
    
        # compare the batch of predictions with the ground truth of labels
        running_corrects += torch.sum(torch.from_numpy(preds) == labels)

    test_acc = running_corrects.double() / len(test_set.dataset)

    print('Acc: {:.2f}%'.format(test_acc*100))

    return test_acc

use_cuda = torch.cuda.is_available()
testacc=test_acc(iter(dataloaders['test']), predictor)
print(testacc)

tensor([ 41,  55, 103,  96, 111])
preds is [  0  55 103   2 111]
tensor([115,  56,  99, 124, 129])
preds is [115  56  81 124 129]
tensor([ 30,  48, 109,  82, 106])
preds is [ 30  48  47  82 106]
tensor([ 94,  70,  48, 103,  73])
preds is [ 94  70  48 103  32]
tensor([ 68, 100, 104,  81,  24])
preds is [ 68 100 104  81  24]
tensor([ 28, 127, 107,  75,  31])
preds is [ 28 127  10  75  31]
tensor([ 39, 116,  46,  14, 123])
preds is [ 39 116  46  14 123]
tensor([78, 67, 19, 80, 86])
preds is [94 67 19 89 86]
tensor([ 9, 45, 35, 64, 79])
preds is [ 7 76 35 64 64]
tensor([ 28,   4,  25,  41, 113])
preds is [ 28   4  25 110 113]
tensor([ 98,  99, 103,  68,  69])
preds is [ 98  81 103  68  69]
tensor([120,  37,  57, 116,  20])
preds is [120  37  57 116  20]
tensor([90, 41, 73, 28, 18])
preds is [90 41 73 28 18]
tensor([ 83,  60,  81, 118,  10])
preds is [114  60  81 113  10]
tensor([100,  40,  15, 109,  35])
preds is [100  40  15 109  35]
tensor([126,  59,  12,  63,  94])
preds is [132  59  12

preds is [85 49 81  9 29]
tensor([78, 81, 18, 58, 86])
preds is [94 81 18 58 86]
tensor([  6,   1, 112,  73,  33])
preds is [ 6  1 94 73 33]
tensor([ 91,  28,  98, 111,  15])
preds is [ 91  28 100  36  15]
tensor([119,  84,   1,  17,  33])
preds is [119  36  35  17  33]
tensor([ 29,  46,  26, 131,  43])
preds is [ 29  46  26 131   7]
tensor([17, 54, 81, 62, 53])
preds is [ 17  46 124  62  53]
tensor([114,  21,  93,   9,  78])
preds is [114  53  93   9  78]
tensor([13, 36, 70, 59, 44])
preds is [13 36 70 59 44]
tensor([  1,  55, 132,  34,  63])
preds is [  1  55 132  34  45]
tensor([72, 53, 19, 77, 55])
preds is [ 62  53   9  77 113]
tensor([106,  89,  37,  72,  90])
preds is [106  89 132 124  90]
tensor([ 30,  52, 123,  16,   6])
preds is [ 30  45 123  16  36]
tensor([  1,   5, 127, 123, 111])
preds is [116   5 127 123 111]
tensor([117,  38, 116,  26,  31])
preds is [117  38 116  26  31]
tensor([41,  8, 11, 88, 37])
preds is [41 34 11 16 37]
tensor([31, 36,  6, 11, 57])
preds is [31 36